In [1]:
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import numpy as np
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization

from keras.utils import np_utils
np.random.seed(1337)  

C:\Users\Dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='coarse')
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [4]:
Xtr = X_train[0:45000]
Ytr = y_train[0:45000]
Xvalid = X_train[45000:]
Yvalid = y_train[45000:]

Xvalid = Xvalid.astype('float32')
X_test = X_test.astype('float32')
Xtr = Xtr.astype('float32')

X_test /= 255.0
Xvalid /= 255.0
Xtr /= 255.0

In [5]:
nb_classes = 20
# Convert class vectors to binary class matrices.
Ytr = np_utils.to_categorical(Ytr, nb_classes)
Yvalid = np_utils.to_categorical(Yvalid, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [6]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same',
input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('elu'))


model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [7]:
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [8]:
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

history = model.fit(Xtr, Ytr,
                    batch_size=32, epochs=15,
                    verbose=1, validation_data=(Xvalid, Yvalid))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
__________

In [9]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.9332347814559936
Test accuracy: 0.6086
